<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from omegaconf import OmegaConf
import torch
from pathlib import Path
from datetime import datetime
import gc, json, os, sys
import cv2
import numpy as np
from tqdm import tqdm
#from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

from stable_diffusion_utils import load_model_from_config, config, generate
from ds_utils.video_utils import imageio_generate_video

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
opt = config()
opt.ckpt = 'D:/models/stable_diffusion/sd-v1-4.ckpt'
opt.config = str(Path.home() / 'Documents/python_workspace/stable-diffusion/configs/stable-diffusion/v1-inference.yaml')
model = load_model_from_config(OmegaConf.load(opt.config), opt.ckpt, opt.full_precision).to(device)

batch_idx = 0
sample_idx = 0

In [ ]:
outputs_path = Path.home() / 'Documents/generated_data/stable_diffusion'
batch_name = f'' # name for subfolder and filenames  {datetime.now().strftime("%Y%m%d_%H%M%S")}_
width_height = [512, 512] 
opt.W, opt.H = width_height
opt.scale = 20 # strength of text prompt
opt.n_steps = 40 # number of diffusion steps
opt.sampler = 'ddim' # ["klms","plms", "ddim"]
opt.ddim_eta = 0.1 # scale of variance from 0.0 to 1.0
opt.seed = -1 # use -1 for random seed or specify number manually
num_batch_images = 3 # how many images you want to generate in this batch

# Init image
opt.init_img = str(Path.home() / '') # url or path to an image, or path to a folder to pick random images from
opt.strength = 0.55 # from 0.0 to 1.0 how much the init image is used

# Mask
opt.mask_path = None #str(Path.home() / '')

# Prompt
opt.prompt = """"""


opt.outdir = str(outputs_path / batch_name)
opt.process_config()

# save settings
settings = vars(opt)
os.makedirs(opt.outdir, exist_ok=True)
while os.path.isfile(f"{opt.outdir}/{batch_name}_{batch_idx}_settings.txt"):
    batch_idx += 1
with open(f"{opt.outdir}/{batch_name}_{batch_idx}_settings.txt", "w+", encoding="utf-8") as f:
    json.dump(settings, f, ensure_ascii=False, indent=4)

sample_idx = 0
# process video
if Path(opt.init_img).suffix in ['.mp4', '.webm']:
    print('Run on video')
    skip_frame = 2
    video = cv2.VideoCapture(opt.init_img)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)/skip_frame

    # while ret:
    frames_out = []
    for frame_index in tqdm(range(frame_count)):
        gc.collect()
        torch.cuda.empty_cache()
        ret, frame = video.read()
        if (frame_index % skip_frame) != 0:
            continue
        if ret:
            opt.init_img = frame
            images = generate(opt, model, batch_name, batch_idx, sample_idx)
            sample_idx += 1
            frames_out.append(images[0][0])
    # generate video
    imageio_generate_video(str(f'{opt.outdir}/{batch_name}_{batch_idx}.mp4'), 
                       [np.array(img) for img in frames_out], fps=fps, format="mp4")
# process image
else:
    for i in range(num_batch_images):
        gc.collect()
        torch.cuda.empty_cache()
        _ = generate(opt, model, batch_name, batch_idx, sample_idx)
        opt.seed += 1
        sample_idx += 1


In [ ]:
from PIL import Image
img = Image.open( str(Path.home() / "")).convert('RGB')
img.thumbnail([600, 600], Image.ANTIALIAS)  # maintain aspect ratio
img.size